### Imports

In [1]:
# Imports
import autogen
import os
import datetime
import json
import time
from dotenv import load_dotenv

In [2]:
# Cell 1: API Configuration, Model Strategy, and Logging Logic

# La funzione per la limitazione del contesto deve essere importata esplicitamente da client_utils.
try:
    from autogen.oai.client_utils import limit_context_window
except ImportError:
    try:
        from autogen.oai.utils import limit_context_window
    except ImportError:
        print("CRITICAL WARNING: Cannot import limit_context_window. Context filtering is DISABLED.")
        def limit_context_window(messages, max_tokens):
            return messages, None
        
# --- Carica Variabili da .env ---
load_dotenv()

# --- 1. API Key Management ---
if "GROQ_API_KEY" not in os.environ:
    raise ValueError("GROQ_API_KEY is required but not found in environment.")

GROQ_BASE_URL = "https://api.groq.com/openai/v1"

# --- 2. Model Leaderboard (Fallback Strategy) ---
config_list = [
    {"model": "openai/gpt-oss-120b", "api_key": os.environ.get("GROQ_API_KEY"), "base_url": GROQ_BASE_URL},
    {"model": "llama-3.3-70b-versatile", "api_key": os.environ.get("GROQ_API_KEY"), "base_url": GROQ_BASE_URL},
    {"model": "mixtral-8x7b-32768", "api_key": os.environ.get("GROQ_API_KEY"), "base_url": GROQ_BASE_URL},
    {"model": "llama-3.1-8b-instant", "api_key": os.environ.get("GROQ_API_KEY"), "base_url": GROQ_BASE_URL}
]

LLM_MODEL_NAME = config_list[0]["model"]

# DEFINIAMO IL LIMITE DI TOKEN UNA SOLA VOLTA
MAX_CONTEXT_TOKENS = 2500

# CONFIGURAZIONE LLM
llm_config = {
    "timeout": 120,
    "config_list": config_list,
    "cache_seed": 42,
    "max_tokens": MAX_CONTEXT_TOKENS,  # Spostato qui dal extra_kwargs
}

# --- 3. Robust Python Logging Function ---
def save_chat_history_to_txt(chat_manager, filename_prefix="execution_log"):
    """
    Extracts chat history and saves it to a clean, readable TXT file.
    Deterministic Python code, no agents involved.
    """
    log_dir = "logs"
    os.makedirs(log_dir, exist_ok=True)
    
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = os.path.join(log_dir, f"{filename_prefix}_{timestamp}.txt")
    
    try:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(f"=== PROJECT FORECASTING LOG - {timestamp} ===\n")
            f.write(f"Model Strategy: Fallback Order {[c['model'] for c in config_list]}\n")
            f.write("="*60 + "\n\n")
            
            messages = list(chat_manager.chat_messages.values())[-1]
            
            for message in messages:
                sender = message.get('name', message.get('role', 'Unknown'))
                content = message.get('content', '')
                
                f.write(f"[{datetime.datetime.now().strftime('%H:%M:%S')}] --- SENDER: {sender.upper()} ---\n")
                f.write("-" * 30 + "\n")
                
                if "function_call" in message and message["function_call"]:
                     f.write(f">> TOOL CALL: {message['function_call'].get('name')}\n")
                     f.write(f">> ARGS: {message['function_call'].get('arguments')}\n")
                
                f.write(content)
                f.write("\n\n" + "="*60 + "\n\n")
                
        print(f"✅ Conversation log successfully saved to: {filename}")
    except Exception as e:
        print(f"❌ Error saving log: {e}")

def create_summary_of_messages(messages):
    """
    Crea un riassunto intelligente dei messaggi mantenendo il contesto critico.
    Mantiene: system messages + riassunto messaggi vecchi + ultimi 5 messaggi completi.
    """
    if len(messages) <= 8:  # Se pochi messaggi, non serve riassunto
        return messages
    
    # Separa system messages dai messaggi normali
    system_msgs = [m for m in messages if m.get("role") == "system"]
    other_msgs = [m for m in messages if m.get("role") != "system"]
    
    if len(other_msgs) <= 5:
        return messages
    
    # Messaggi da riassumere (tutti tranne gli ultimi 5)
    to_summarize = other_msgs[:-5]
    recent_msgs = other_msgs[-5:]
    
    # Crea riassunto testuale
    summary_text = "=== CONVERSATION SUMMARY ===\n"
    for msg in to_summarize:
        role = msg.get("role", "unknown")
        name = msg.get("name", role)
        content = msg.get("content", "")[:150]  # Max 150 char per messaggio
        summary_text += f"[{name}]: {content}...\n"
    
    summary_text += "=== END SUMMARY ===\n"
    
    # Crea messaggio di riassunto
    summary_message = {
        "role": "system",
        "content": summary_text
    }
    
    # Ritorna: system messages + summary + ultimi 5 messaggi
    return system_msgs + [summary_message] + recent_msgs

def smart_message_transform(messages):
    """
    Trasforma i messaggi per ridurre i token mantenendo il contesto.
    Conta token approssimativi e riassume se necessario.
    """
    # Conta token approssimativi (4 caratteri = 1 token circa)
    total_chars = sum(len(str(m.get("content", ""))) for m in messages)
    estimated_tokens = total_chars / 4
    
    # Se siamo sopra ~3500 token stimati, riassumi
    if estimated_tokens > 3500:
        print(f"⚠️ Token Warning: ~{int(estimated_tokens)} tokens detected. Summarizing old messages...")
        return create_summary_of_messages(messages)
    
    return messages

CRITICAL WARNING: Cannot import limit_context_window. Context filtering is DISABLED.


In [3]:
# Cell 2: Global Variables and Admin Agent Setup

# --- Global Configuration Variables ---
# Safety and Control - MANTIENI I TUOI VALORI
ASK_CODE_PERMISSION = True   # ✅ MANTIENI: Chiedi conferma
CAN_ASK_HUMAN = True          # ✅ MANTIENI: Puoi intervenire

# Trading Strategy Parameters
TRADING_COMMISSION_PCT = 0.00055
SLIPPAGE_K = 0.01
INITIAL_CAPITAL_USDT = 10000.0

# Time Constraints (UTC)
OPEN_TIME_UTC_START = 11
OPEN_TIME_UTC_END = 19
MAX_PREDICTION_WINDOW_MINUTES = 23 * 60 + 59

# --- Admin Agent Definition ---
admin = autogen.UserProxyAgent(
    name="Admin",
    system_message="Execute code after user approval.",
    human_input_mode="ALWAYS",
    code_execution_config={
        "work_dir": ".",
        "use_docker": False,
    },
    max_consecutive_auto_reply=999,
)

In [4]:
def create_assistant_agent(name, role_description):
    """
    Crea un AssistantAgent ottimizzato per ESEGUIRE codice, non solo discuterne.
    """
    common_instruction = (
        f"Using Groq API ({LLM_MODEL_NAME}). "
        "CRITICAL: Write ```python code blocks that Admin can execute immediately. "
        "DO NOT just discuss code - PROPOSE it for execution. "
        "BE CONCISE: max 3 sentences + code block. "
        "Use CPU-only models (RandomForest/XGBoost). "
    )
    
    # Funzione per terminare su messaggi vuoti (evita loop)
    def is_termination_msg(content):
        if content is None:
            return True
        msg_content = content.get("content", "")
        if msg_content.strip() == "":
            return True
        return False
    
    return autogen.AssistantAgent(
        name=name,
        llm_config=llm_config,
        system_message=f"{role_description} {common_instruction}",
        is_termination_msg=is_termination_msg,
        max_consecutive_auto_reply=2,
    )

# --- Define the Specialists ---
architect = create_assistant_agent(
    "Project_Architect", 
    "You are an expert in robust project structuring. Your first task is to create the folder structure (src, data/raw, data/processed, models, testing) and a comprehensive config.py file based on all initial parameters."
)

data_scraper = create_assistant_agent(
    "Data_Scraper", 
    "You are the world's expert in data cleaning and integration. Your task is to **load ALL local CSV files** from the 'data/raw' folder, perform cleaning (NaN, missing data), time-series synchronization (using the 'timestamp' column), and feature generation, preparing a single clean DataFrame for the Model_Builder. **DO NOT fetch data from the web.**"
)

model_builder = create_assistant_agent(
    "Model_Builder", 
    "You are the top Data Scientist for time series. Perform feature engineering, train a robust **CPU-compatible model (e.g., RandomForest, XGBoost, or simple Scikit-learn models)** to predict maximum price change, and save the final model."
)

optimizer = create_assistant_agent(
    "Optimizer_Simulator", 
    "You are the Financial Risk and Optimization expert. Create the trading simulation (100/L - 0.5% liquidation logic) and optimize Integer Leverage (1-100), Stop Loss, and Take Profit to maximize log_growth."
)

code_tester = create_assistant_agent(
    "Code_Tester", 
    "You are the Production QA Engineer. Stress-test the code, ensure robust NaN error handling, and provide the final inference code formatted as requested."
)

# --- Custom Manager con Message Transform ---
class SummarizingGroupChatManager(autogen.GroupChatManager):
    """
    GroupChatManager che riassume automaticamente i messaggi lunghi.
    """
    def _process_received_message(self, message, sender, silent):
        # Prima di processare, applica il transform
        if hasattr(self, 'groupchat') and hasattr(self.groupchat, 'messages'):
            self.groupchat.messages = smart_message_transform(self.groupchat.messages)
        
        # Chiama il metodo originale
        return super()._process_received_message(message, sender, silent)

# --- Group Chat Setup CON TUTTI GLI AGENTI ---
agent_list = [admin, architect, data_scraper, model_builder, optimizer, code_tester]

group_chat = autogen.GroupChat(
    agents=agent_list, 
    messages=[], 
    max_round=30,  # ⚠️ RIDOTTO da 50 a 30 per evitare loop troppo lunghi
    speaker_selection_method="auto",
)

# Usa il manager custom con summarization
manager = SummarizingGroupChatManager(
    groupchat=group_chat, 
    llm_config=llm_config
)

In [ ]:
# Cell 4: Initiate Process and Save Log

MAIN_TARGET_FILE = "Bitcoin futures (USDT) 25-03-2020-10-36-00_07-12-2025-00-00-00 timeframe 1m.csv"
DATA_FOLDER = "data/raw" 

print("=" * 60)
print("ESECUZIONE SEQUENZIALE CON CONFERMA UTENTE")
print("=" * 60)

# ===== FASE 1: Setup Progetto =====
print("\n[FASE 1/5] Setup Progetto")
print("-" * 60)
phase1_result = admin.initiate_chat(
    architect,
    message=(
        "Create folders: src, data/raw, data/processed, models, testing. "
        "Create src/config.py that loads GROQ_API_KEY with os.getenv(). "
        "Write EXECUTABLE Python code in ```python blocks."
    ),
    # ⚠️ AUMENTATO max_turns per permettere conferma + esecuzione
    max_turns=5
)

# ===== FASE 2: Creazione del TARGET =====
print("\n[FASE 2/5] Creazione Target Variable")
print("-" * 60)
phase2_result = admin.initiate_chat(
    data_scraper,
    message=(
        f"CRITICAL TASK: Create the target variable from '{MAIN_TARGET_FILE}'. "
        "The target is: maximum absolute price change (for example +2000 USDT or -2500 USDT) from 11:00 UTC each day to 10:59 UTC of the day after. "
        "For each day: "
        "1. Calculate: max_change = max(close) - min(close) for that window "
        "2. Store as 'target_max_variation' column "
        "Save to data/processed/target_data.csv. "
        "Write EXECUTABLE Python code."
    ),
    max_turns=5
)

# ===== FASE 3: Feature Engineering =====
print("\n[FASE 3/5] Feature Engineering")
print("-" * 60)
phase3_result = admin.initiate_chat(
    data_scraper,
    message=(
        f"Load ALL CSV files from '{DATA_FOLDER}'. "
        "Clean NaN, synchronize timestamps. "
        "Merge with target_data.csv created previously. "
        "Create features: lags, moving averages, volatility, volume metrics. "
        "Save final dataset to data/processed/final_dataset.csv. "
        "Write EXECUTABLE Python code."
    ),
    max_turns=5
)

# ===== FASE 4: Model Training =====
print("\n[FASE 4/5] Model Training")
print("-" * 60)
phase4_result = admin.initiate_chat(
    model_builder,
    message=(
        "Load data/processed/final_dataset.csv. "
        "Target column: 'target_max_variation'. "
        "Train CPU-compatible model (RandomForest or XGBoost). "
        "Use train_test_split. "
        "Save trained model to models/best_model.pkl. "
        "Write EXECUTABLE Python code."
    ),
    max_turns=5
)

# ===== FASE 5: Backtesting =====
print("\n[FASE 5/5] Backtesting & Optimization")
print("-" * 60)
phase5_result = admin.initiate_chat(
    optimizer,
    message=(
        "Load model from models/best_model.pkl. "
        "Run backtest with liquidation logic: 100/L - 0.5%. "
        f"Initial capital: {INITIAL_CAPITAL_USDT} USDT. "
        f"Commission: {TRADING_COMMISSION_PCT}. "
        "Optimize Leverage (1-100), SL, TP to maximize log_growth. "
        "Save optimal parameters to models/optimal_params.json. "
        "Write EXECUTABLE Python code."
    ),
    max_turns=5
)

print("\n" + "=" * 60)
print("TUTTE LE FASI COMPLETATE")
print("=" * 60)

# --- Save Log ---
save_chat_history_to_txt(manager, filename_prefix="BTC_Sequential_WithTarget")

ESECUZIONE SEQUENZIALE CON CONFERMA UTENTE

[FASE 1/5] Setup Progetto
------------------------------------------------------------
Admin (to Project_Architect):

Create folders: src, data/raw, data/processed, models, testing. Create src/config.py that loads GROQ_API_KEY with os.getenv(). Write EXECUTABLE Python code in ```python blocks.

--------------------------------------------------------------------------------


[autogen.oai.client: 12-12 14:46:28] {732} WARNING - Model openai/gpt-oss-120b is not found. The cost will be 0. In your config_list, add field {"price" : [prompt_price_per_1k, completion_token_price_per_1k]} for customized pricing.
Project_Architect (to Admin):

Here’s a single script you can run now; it creates the required folder tree and writes a `src/config.py` that pulls the `GROQ_API_KEY` from the environment.

```python
import os
from pathlib import Path

# 1️⃣ Define the folder layout
folders = [
    Path("src"),
    Path("data/raw"),
    Path("data/processed"),
    Path("models"),
    Path("testing"),
]

# 2️⃣ Create all missing directories (parents=True handles nested paths)
for f in folders:
    f.mkdir(parents=True, exist_ok=True)

# 3️⃣ Write src/config.py with a safe env‑var loader
config_path = Path("src/config.py")
config_path.write_text(
    """import os

# Load the Groq API key from the environment; raise a clear error if missing
GROQ_API_KEY = os.getenv("GROQ_API_KE